In [53]:
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
from keras.layers import Activation, Dense, Dropout, Flatten
from tensorflow.keras.layers import BatchNormalization
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tfs
from keras.layers import Activation, Dense, Dropout, Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model  # Import Model from tensorflow.keras.models
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab


In [54]:
# Get images
image = img_to_array(load_img('woman.jpg'))
image = np.array(image, dtype=float)

In [55]:
X = rgb2lab(1.0/255*image)[:,:,0]
Y = rgb2lab(1.0/255*image)[:,:,1:]
Y /= 128
X = X.reshape(1, 400, 400, 1)
Y = Y.reshape(1, 400, 400, 2)



model = Sequential()
model.add(InputLayer(input_shape=(None, None, 1)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

In [56]:
from tensorflow.keras.layers import Input, Conv2D, UpSampling2D
from tensorflow.keras.models import Model

# Define the input layer
input_layer = Input(shape=(None, None, 1))

# Convolutional Layers
conv1 = Conv2D(8, (3, 3), activation='relu', padding='same', strides=2)(input_layer)
conv2 = Conv2D(8, (3, 3), activation='relu', padding='same')(conv1)
conv3 = Conv2D(16, (3, 3), activation='relu', padding='same')(conv2)
conv4 = Conv2D(16, (3, 3), activation='relu', padding='same', strides=2)(conv3)
conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv4)
conv6 = Conv2D(32, (3, 3), activation='relu', padding='same', strides=2)(conv5)

# Upsampling Layers
up1 = UpSampling2D((2, 2))(conv6)
conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(up1)
up2 = UpSampling2D((2, 2))(conv7)
conv8 = Conv2D(16, (3, 3), activation='relu', padding='same')(up2)
up3 = UpSampling2D((2, 2))(conv8)

# Final Convolutional Layer
output_layer = Conv2D(2, (3, 3), activation='tanh', padding='same')(up3)

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

# Print a summary of the model architecture
model.summary()


Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, None, None, 1)]   0         
                                                                 
 conv2d_36 (Conv2D)          (None, None, None, 8)     80        
                                                                 
 conv2d_37 (Conv2D)          (None, None, None, 8)     584       
                                                                 
 conv2d_38 (Conv2D)          (None, None, None, 16)    1168      
                                                                 
 conv2d_39 (Conv2D)          (None, None, None, 16)    2320      
                                                                 
 conv2d_40 (Conv2D)          (None, None, None, 32)    4640      
                                                                 
 conv2d_41 (Conv2D)          (None, None, None, 32)    9248

In [58]:
# Finish model
model.compile(optimizer='adam', loss='mean_squared_error')


In [59]:
model.fit(x=X, 
    y=Y,
    batch_size=1,
    epochs=10)

Epoch 1/10
1/1 [==============================] - 4s 4s/step - loss: 0.1032
Epoch 2/10
1/1 [==============================] - 0s 359ms/step - loss: 0.0379
Epoch 3/10
1/1 [==============================] - 0s 338ms/step - loss: 0.0142
Epoch 4/10
1/1 [==============================] - 0s 323ms/step - loss: 0.0092
Epoch 5/10
1/1 [==============================] - 0s 346ms/step - loss: 0.0085
Epoch 6/10
1/1 [==============================] - 0s 378ms/step - loss: 0.0083
Epoch 7/10
1/1 [==============================] - 0s 335ms/step - loss: 0.0081
Epoch 8/10
1/1 [==============================] - 0s 367ms/step - loss: 0.0079
Epoch 9/10
1/1 [==============================] - 0s 348ms/step - loss: 0.0076
Epoch 10/10
1/1 [==============================] - 0s 327ms/step - loss: 0.0074


In [64]:
import cv2
print(model.evaluate(X, Y, batch_size=1))
output = model.predict(X)
output *= 128
# Output colorizations
cur = np.zeros((2048,1605, 3))
gray_image = X[0][:,:,0]
resized_gray_image = cv2.resize(gray_image, (1605, 2048))
cur[:,:,0] = resized_gray_image
#cur[:,:,0] = X[0][:,:,0]
cur[:,:,1:] = output[0]
imsave("img_result.png", lab2rgb(cur))
imsave("img_gray_version.png", rgb2gray(lab2rgb(cur)))


import cv2

# Load the grayscale image
gray_image = X[0][:,:,0]

# Resize the grayscale image to match the dimensions of cur
resized_gray_image = cv2.resize(gray_image, (2048, 1605))

# Assign the resized grayscale image to the first channel of cur
cur[:,:,0] = resized_gray_image.T  # Transpose the image






# Assign the resized grayscale image to the first channel of cur
cur[:,:,0] = resized_gray_image


1/1 [==============================] - 0s 115ms/step - loss: 0.0072
0.007248362060636282
1/1 [==============================] - 0s 126ms/step


ValueError: could not broadcast input array from shape (400,400,2) into shape (2048,1605,2)